In [3]:
import optuna
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Load the local dataset
file_paths = [
    "C:/VS code projects/data_files/Monday-WorkingHours.pcap_ISCX.csv",
    "C:/VS code projects/data_files/Tuesday-WorkingHours.pcap_ISCX.csv",
    "C:/VS code projects/data_files/Wednesday-workingHours.pcap_ISCX.csv",
    "C:/VS code projects/data_files/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv",
    "C:/VS code projects/data_files/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv",
    "C:/VS code projects/data_files/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv",
    "C:/VS code projects/data_files/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv",
    "C:/VS code projects/data_files/Friday-WorkingHours-Morning.pcap_ISCX.csv"
]



# Read and clean datasets
dataframes = []
for file_path in file_paths:
    df = pd.read_csv(file_path)
    df.columns = df.columns.str.strip()  # Remove whitespace from column names
    dataframes.append(df)

# Combine all datasets into a single DataFrame
df = pd.concat(dataframes, ignore_index=True)
df['Label'] = df['Label'].apply(lambda x: 1 if x == 'BENIGN' else 0)


In [ ]:

# Find the minimum class count
min_count = df['Label'].value_counts().min()


# Perform undersampling to balance the dataset
df = df.groupby('Label', group_keys=False).apply(lambda x: x.sample(n=min_count, random_state=42, ignore_index=True))

# Preprocessing dataset
print("Preprocessing dataset...")
df.dropna(inplace=True)  # Remove missing values
df.replace([np.inf, -np.inf], np.nan, inplace=True)  # Replace infinite values with NaN
df.columns = df.columns.str.strip()  # Strip whitespaces from column names

# Separate features and labels
X = df.drop('Label', axis=1)  # Features
Y = df['Label']              # Labels

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)


# Define the objective function for Optuna
def objective(trial):
    # Define the hyperparameter search space
    param_grid = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 10.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
    }

    # Train LightGBM with the current set of hyperparameters
    model = lgb.LGBMClassifier(**param_grid, random_state=42)  # Unpack param_grid using **
    model.fit(
        X_train, 
        y_train, 
        eval_set=[(X_test, y_test)], 
        eval_metric='logloss', 
        callbacks=[lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(0)]  # Use callbacks for verbosity and early stopping
    )

    # Predict on the validation set
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    # Return the accuracy as the objective to maximize
    return accuracy

# Create a study and optimize the objective function
study = optuna.create_study(direction='maximize')  # Maximize accuracy
study.optimize(objective, n_trials=50, timeout=3600)  # Run 50 trials or stop after 1 hour

# Print the best hyperparameters
print("Best Hyperparameters:")
print(study.best_params)

C:\Users\Spencer Kanjera\AppData\Local\Temp\ipykernel_1724\2686234378.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Label', group_keys=False).apply(lambda x: x.sample(n=min_count, random_state=42, ignore_index=True))


Preprocessing dataset...


[I 2025-04-11 13:46:00,157] A new study created in memory with name: no-name-a8cda311-d37d-4399-9ce9-8b085d5db084
C:\Users\Spencer Kanjera\AppData\Local\Temp\ipykernel_1724\2686234378.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
C:\Users\Spencer Kanjera\AppData\Local\Temp\ipykernel_1724\2686234378.py:30: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
C:\Users\Spencer Kanjera\AppData\Local\Temp\ipykernel_1724\2686234378.py:31: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optu

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 390213, number of negative: 389764
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.293153 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14459
[LightGBM] [Info] Number of data points in the train set: 779977, number of used features: 70
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500288 -> initscore=0.001151
[LightGBM] [Info] Start training from score 0.001151
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

[I 2025-04-11 13:46:12,327] Trial 0 finished with value: 0.9968140099797772 and parameters: {'learning_rate': 0.018686242954017208, 'num_leaves': 62, 'max_depth': 7, 'min_child_samples': 58, 'subsample': 0.682740307829313, 'colsample_bytree': 0.5837387648325969, 'reg_alpha': 1.0655267559717458e-08, 'reg_lambda': 0.11159034236590439, 'n_estimators': 108}. Best is trial 0 with value: 0.9968140099797772.
C:\Users\Spencer Kanjera\AppData\Local\Temp\ipykernel_1724\2686234378.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
C:\Users\Spencer Kanjera\AppData\Local\Temp\ipykernel_1724\2686234378.py:30: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use

...

[I 2025-04-11 14:07:27,728] Trial 48 finished with value: 0.9994256243343823 and parameters: {'learning_rate': 0.1743732713927286, 'num_leaves': 97, 'max_depth': 8, 'min_child_samples': 60, 'subsample': 0.9298406571270155, 'colsample_bytree': 0.5457204350501974, 'reg_alpha': 0.0005158204289502126, 'reg_lambda': 0.06175921688560675, 'n_estimators': 393}. Best is trial 29 with value: 0.9994495566537831.
C:\Users\Spencer Kanjera\AppData\Local\Temp\ipykernel_1724\2686234378.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
C:\Users\Spencer Kanjera\AppData\Local\Temp\ipykernel_1724\2686234378.py:30: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 390213, number of negative: 389764
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.348556 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14459
[LightGBM] [Info] Number of data points in the train set: 779977, number of used features: 70
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500288 -> initscore=0.001151
[LightGBM] [Info] Start training from score 0.001151
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

[I 2025-04-11 14:07:47,797] Trial 49 finished with value: 0.9994076750948319 and parameters: {'learning_rate': 0.1251788831629756, 'num_leaves': 50, 'max_depth': 6, 'min_child_samples': 33, 'subsample': 0.8863296006443722, 'colsample_bytree': 0.6532936068784452, 'reg_alpha': 2.1258625956425336e-06, 'reg_lambda': 0.001952309322716981, 'n_estimators': 226}. Best is trial 29 with value: 0.9994495566537831.


Best Hyperparameters:
{'learning_rate': 0.06486348591551848, 'num_leaves': 130, 'max_depth': 11, 'min_child_samples': 64, 'subsample': 0.6848539751617823, 'colsample_bytree': 0.5005320541074223, 'reg_alpha': 1.6363250831261822e-08, 'reg_lambda': 8.631874061803956e-05, 'n_estimators': 324}


: 